# Dataset Summarize From Feedback

Take care of all the data cleaning to get the formatting to follow MT-Bench standards.

In [1]:
import pandas as pd
import plotly as ply
import numpy as np
import os
from tqdm import tqdm
import pickle

In [2]:
from datasets import load_dataset, load_dataset_builder

In [4]:
ds_builder = load_dataset_builder("tatsu-lab/alpaca_farm", "alpaca_gpt4_preference")

In [5]:
ds_builder.info.description

'\nData used in the original AlpacaFarm experiments.\nIncludes SFT and preference examples.\n'

In [6]:
ds_builder.info.features

{'instruction': Value(dtype='string', id=None),
 'input': Value(dtype='string', id=None),
 'output_1': Value(dtype='string', id=None),
 'output_2': Value(dtype='string', id=None),
 'preference': Value(dtype='int64', id=None)}

In [7]:
train_data = load_dataset("tatsu-lab/alpaca_farm", "alpaca_gpt4_preference", split ='preference[0:5000]')

/Users/johnwang/.cache/huggingface/datasets/downloads/4d9842f56ccb867b37940153b8fdeb286cf947e72dfc1377685f50ff22f4ae6c


Generating preference split: 0 examples [00:00, ? examples/s]

Dataset alpaca_farm downloaded and prepared to /Users/johnwang/.cache/huggingface/datasets/tatsu-lab___alpaca_farm/alpaca_gpt4_preference/1.0.0/79d38dc3f12abd62869e376303b68092e8385769e22f05166fe96a3dac29a57a. Subsequent calls will reuse this data.


In [8]:
train_data

Dataset({
    features: ['instruction', 'input', 'output_1', 'output_2', 'preference'],
    num_rows: 5000
})

In [9]:
df = train_data.to_pandas()

In [13]:
df.shape

(5000, 5)

In [14]:
df.head()

,instruction,input,output_1,output_2,preference
0,Access the default browser settings on a compu...,,To access the default browser settings on a co...,"In the Google Chrome browser, the default sett...",2
1,Access the third element of the list.,"[0, 1, 6, 3, 4]",The third element of the list is 3.,3,2
2,"According to the given text, what is the main ...","As technology continues to develop and evolve,...",The main argument being made is that technolog...,The main argument being made is that technolog...,2
3,"According to the source article, what is the p...",,The purpose of writing a press release is to d...,The purpose of writing a press release is to c...,2
4,"According to the story, what is the main messa...","Doha, Qatar (CNN) — On the eve of the 2022 Wor...",The main message that FIFA President Gianni In...,The main message that FIFA President Gianni In...,2


# Create Questions JSONL: (sff_questions.jsonl)

In [124]:
questions = df[['question_id', 'category', 'turns']].copy()
questions.head()

,question_id,category,turns
0,0,summary,SUBREDDIT: relationships TITLE: Mother [51] no...
1,1,summary,SUBREDDIT: relationships TITLE: Mother [51] no...
2,2,summary,SUBREDDIT: relationships TITLE: Mother [51] no...
3,3,summary,SUBREDDIT: relationships TITLE: Mother [51] no...
4,4,summary,SUBREDDIT: AskReddit TITLE: Can I sue my prope...


In [125]:
questions['turns'] = 'Given the context of the specified subreddit and title, summarize the post. ' + questions['turns']
questions['turns'] = questions['turns'].apply(lambda x: [x])

In [130]:
q_json = questions.to_json(orient='records')
#q_json

In [134]:
import json
with open("sff_test.jsonl", "w") as file:
    for item in questions.to_dict('records'):
        json_line = json.dumps(item)
        file.write(json_line + '\n')